In [9]:
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support
from torch.optim.lr_scheduler import StepLR

In [10]:
# Veri seti yolunu belirleyin
train_dir = 'D:/bitirme/fer_dataset/train'
test_dir = 'D:/bitirme/fer_dataset/test'

In [11]:
# Veri dönüşümleri ve veri artırma
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(48, scale=(0.8, 1.0)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]),
}

In [12]:
# Veri seti yükleme
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [13]:
# Sınıf isimleri
class_names = train_dataset.classes

In [16]:
# Model tanımlama (EfficientNet)
model = models.efficientnet_b0(pretrained=True)
model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)  # Giriş katmanını gri tonlamalı resimlere uyarlamak
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_names))  # Çıkış katmanını sınıf sayısına uyarlamak

In [17]:
# Cihaz belirleme (GPU kullanımı için)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [18]:
# Kayıp fonksiyonu ve optimizasyon algoritması
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

In [19]:
# Eğitim fonksiyonu
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_f1 = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        scheduler.step()

        epoch_loss = running_loss / len(train_dataset)
        print(f'Loss: {epoch_loss:.4f}')

        # Performans değerlendirmesi
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        precision, recall, f1_score, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}')

        # En iyi modeli kaydetme
        if f1_score > best_f1:
            best_f1 = f1_score
            best_model_wts = copy.deepcopy(model.state_dict())

    print(f'En iyi F1-Score: {best_f1:.4f}')
    model.load_state_dict(best_model_wts)
    return model

In [20]:
# Modeli eğit
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 1/25
----------
Loss: 1.5575
Precision: 0.4398, Recall: 0.4733, F1-Score: 0.4274
Epoch 2/25
----------
Loss: 1.2807
Precision: 0.5191, Recall: 0.5285, F1-Score: 0.5082
Epoch 3/25
----------
Loss: 1.1790
Precision: 0.5639, Recall: 0.5376, F1-Score: 0.5256
Epoch 4/25
----------
Loss: 1.1165
Precision: 0.5883, Recall: 0.5786, F1-Score: 0.5723
Epoch 5/25
----------
Loss: 1.0640
Precision: 0.6064, Recall: 0.6027, F1-Score: 0.5977
Epoch 6/25
----------
Loss: 1.0257
Precision: 0.6125, Recall: 0.6143, F1-Score: 0.6114
Epoch 7/25
----------
Loss: 0.9941
Precision: 0.6042, Recall: 0.5991, F1-Score: 0.5890
Epoch 8/25
----------
Loss: 0.8945
Precision: 0.6295, Recall: 0.6327, F1-Score: 0.6269
Epoch 9/25
----------
Loss: 0.8536
Precision: 0.6266, Recall: 0.6328, F1-Score: 0.6270
Epoch 10/25
----------
Loss: 0.8300
Precision: 0.6316, Recall: 0.6387, F1-Score: 0.6315
Epoch 11/25
----------
Loss: 0.8056
Precision: 0.6334, Recall: 0.6375, F1-Score: 0.6328
Epoch 12/25
----------
Loss: 0.7877
Preci

In [25]:
torch.save(model.state_dict(), 'emotion_recognition_model.pth')

In [26]:
# Modelin performansını test etme
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [28]:
# Precision, Recall, F1-Score hesaplama
precision, recall, f1_score, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')
print(f'Test F1-Score: {f1_score:.4f}')

Test Precision: 0.6463
Test Recall: 0.6495
Test F1-Score: 0.6448
